In [1]:
from dotenv import load_dotenv
from openai import OpenAI
import os

load_dotenv(override=True)  # This loads the environment variables from .env file
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

In [2]:
def QnA(
    art,
    client,
    model,
    QnA_prompt,
):

    prompt_complete = QnA_prompt + "\n" + art["body"]
    model = model
    messages = [{"role": "user", "content": prompt_complete}]
    try:
        response = client.chat.completions.create(
            model=model, messages=messages, temperature=0
        )

        content = response.choices[0].message.content
        return content
    except Exception as e:  # if the model fails to return a response
        print(f"Error: {e}")
        return "Sorry, error from GPT."

In [3]:
import json


def get_LLM_QnA(
    src_filename,
    client,
    model="gpt-4o",
    QnA_prompt="",
):
    import pandas as pd

    # read the label file
    labels_T = pd.read_csv("../CA/labels_4o.csv", header=None, names=["label"])
    labels_T = labels_T.astype(int)
    # get the label index if labe is 1
    label_index = list(labels_T[labels_T["label"] == 1].index)
    label_index_zho = [i - 100 for i in label_index if i >= 100]

    with open(src_filename, "r") as f:
        data = f.read()
        des_articles = []
        src_articles = json.loads(data)

        for i in label_index_zho:
            art = {}
            art["body_eng"] = QnA(
                src_articles[i],
                client,
                model=model,
                QnA_prompt=QnA_prompt,
            )

            des_articles.append(art)

    des_filename = "eng_" + src_filename
    if os.path.exists(des_filename):
        os.remove(des_filename)
    with open(des_filename, "w") as f:
        json.dump(des_articles, f, ensure_ascii=False, indent=4)

In [4]:
question = "You are an expert at translation. Please translate the following news article from Chinese to English."
get_LLM_QnA(
    "narrative_QnA_summary_Russo_Ukrainian_War_zho.json",
    client,
    model="gpt-4o",
    QnA_prompt=question,
)